<a href="https://colab.research.google.com/github/Aliaksandr-Borsuk/Recommender_Systems_project/blob/main/notebooks/03_01_matrix_factorization_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка


**Цель:**
- построить и сравнить модели матричной факторизации, подобрать параметры, оценить на валидации и тесте   
**Данные:**
- используем train(разбиение на v_train, val), test  из предыдущих ноутбуков.
- **############# TODO ########################**

## 01. Клонируем репозиторий.Подключаем GoogleDrive.

In [1]:
!rm -rf /content/Recommender_Systems_project
!git clone https://github.com/Aliaksandr-Borsuk/Recommender_Systems_project

Cloning into 'Recommender_Systems_project'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 97 (delta 41), reused 45 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (97/97), 548.39 KiB | 5.32 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [2]:
# подключаем диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=10790422 sha256=1fa55ce930395395805ec3cf6b2c64cb6dc9884a09e2d2ccfdf9a13aa2367a53
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 7.9 MB/s eta 0:00:00


## 02. Импорты

In [4]:
import sys
sys.path.append("/content/Recommender_Systems_project/src")

import optuna
from optuna.samplers import TPESampler

import time
import pickle
from sklearn.decomposition import TruncatedSVD
# from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import save_npz, load_npz, csr_matrix
from pprint import pprint
from pathlib import Path
import pandas as pd
import numpy as np
import random

# импортируем нужные функци
from recommender.data_io import train_test_reader # для чтения сохранённых из 001_data_and_eda_1m_proba
from recommender.splitters import df_time_split  # для time_split разбиения df
from recommender.preprocessing import prepare_ui_matrix  # для получения матрицы взаимодействий
from recommender.metrics import hitrate_at_k, coverage_at_k, precision_at_k,\
                     recall_at_k, ndcg_at_k, map_at_k, model_evaluation
from recommender.results_logger import save_experiment_results # для сохранения результатов



RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

DATA =   Path("/content/drive/MyDrive/Colab Notebooks/data/")
RAW_DATA = DATA/"raw"
PROCESSED = DATA/"processed"
RESULTS_DIR = DATA / "results"
DATA.mkdir(exist_ok=True)
TOP_N = 10

## 03. Грузим train, test, meta_данные.



In [5]:
train_tast_path = '/content/drive/MyDrive/Colab Notebooks/data/processed/251021_173655'

train, test, meta = train_test_reader(train_tast_path)
pprint(meta, width=80, compact=False)
print(f'\ntrain shape : {train.shape}')
print(f'test shape  : {test.shape}')
print( '\n', '*'*50, '\ntrain.head')
display(train.head(3))
print('\n', '*'*50, '\ntest.head')
display(test.head(3))

{'columns': ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres'],
 'created_at': '2025-10-21T17:37:00.607645',
 'min_test_interactions': 10,
 'min_train_interactions': 5,
 'n_items': 3662,
 'n_test_users': 836,
 'n_train_users': 5392,
 'test_shape': [94842, 6],
 'time_treshold': '2000-12-02T14:52:18',
 'train_shape': [800142, 6]}

train shape : (800142, 6)
test shape  : (94842, 6)

 ************************************************** 
train.head


,user_id,item_id,rating,timestamp,title,genres
0,635,1251,4,975768620,8 1/2 (1963),Drama
1,635,3948,4,975768294,Meet the Parents (2000),Comedy
2,635,1270,4,975768106,Back to the Future (1985),Comedy|Sci-Fi



 ************************************************** 
test.head


,user_id,item_id,rating,timestamp,title,genres
0,635,3789,5,975768788,"Pawnbroker, The (1965)",Drama
1,635,2987,5,979141847,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
2,635,2988,4,975769007,Melvin and Howard (1980),Drama


## 04. Бьем Train на v_train и val для подбора гиперпараметров
Используем опять вариант **B** - разбиение по времени(Time-based split).
 Все события до порога в v_train, события после в val. Это гарантирует, что ни одно событие из "будущего" не попадёт в v_train.

In [6]:
# Бьем Train на v_train и val
# колонки для сохранения
columns_to_save = ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres']
user_id = 'user_id'
item_id = 'item_id'
time_column = 'timestamp'

# минимальное тёплое число оценок
min_n_reitings = 5
# min кол-во оценок у каждого юзера в train и в test
n, k = 15, 10
quantile = 0.85    # квантиль разбиения
# разбиение
v_train, val = df_time_split(train, time_column, columns_to_save,
                             user_id , item_id,
                             min_n_reitings, n, k , quantile)

users_intersection = set(val['user_id']) & set(v_train['user_id'])
print('*'*70)
print(f"\n{len(users_intersection)} юзера встречаются одновременно и в v_train и в val")

# залистим тестовых пользователей
val_users = sorted(users_intersection)

print('\ntrain ')
display(v_train.head(2))
print('\ntest ')
display(val.head(2))

Порог разбиения по времени 974814617
Размеры после разбиения: train 680122 test 120020

Осталось пользователей в train : 4748
Новый размер train: (680108, 6)
Осталось пользователей в test: 211
Новый размер test: (26076, 6)
**********************************************************************

211 юзера встречаются одновременно и в v_train и в val

train 


,user_id,item_id,rating,timestamp,title,genres
0,1272,720,5,974814383,Wallace & Gromit: The Best of Aardman Animatio...,Animation
1,1272,2993,2,974814617,Thunderball (1965),Action



test 


,user_id,item_id,rating,timestamp,title,genres
0,1272,2987,4,974822814,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
1,1272,2989,4,974814713,For Your Eyes Only (1981),Action


## 05. подбор гиперпараметров Optuna
- т.к. используем ту же матрицу (по форме) для обучения то и словарики можно брать старые

In [7]:
# Загрузка словарей
input_dir = PROCESSED/"artifacts"

with open(input_dir / "user2index_v.pkl", "rb") as f:
    user2index_v = pickle.load(f)

with open(input_dir / "item2index_v.pkl", "rb") as f:
    item2index_v = pickle.load(f)

with open(input_dir / "index2user_v.pkl", "rb") as f:
    index2user_v = pickle.load(f)

with open(input_dir / "index2item_v.pkl", "rb") as f:
    index2item_v = pickle.load(f)

In [8]:
# заменяем реальные ID на индексы из explicit-матрицы обучения модели
val_mapped_ex = val.assign(
    user_id = val["user_id"].map(user2index_v),
    item_id = val["item_id"].map(item2index_v)
)
assert val_mapped_ex.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
val_items_ex = val_mapped_ex.groupby('user_id')['item_id'].apply(set).to_dict()

#все айтемы
all_items_ex = set(v_train['item_id'].map(item2index_v).dropna().astype(int).unique())

In [9]:
def recommend_with_svd(svd_model, train_matrix, val_users, topn=10) -> dict[int, list[int]]:
    """
    Генерация рекомендаций для всех пользователей на основе SVD.
    """
   # Эмбеддинги пользователей - только для нужных пользователей
    X_svd = svd_model.transform(train_matrix[val_users])  # (n_val_users, n_components)

    components_ = svd_model.components_  # (n_components, n_items)

    # Матричное умножение для всех пользователей сразу
    user_scores = X_svd @ components_  # (n_val_users, n_items)

    # Маска уже просмотренных айтемов
    seen_mask = train_matrix[val_users].toarray().astype(bool)  # (n_val_users, n_items)
    user_scores[seen_mask] = -np.inf

    # Получаем топ-N индексов для всех пользователей сразу
    top_indices = np.argsort(-user_scores, axis=1)[:, :topn]  # (n_val_users, topn)

    # Создаем словарь рекомендаций
    recs = {user_id: indices.tolist() for user_id, indices in zip(val_users, top_indices)}

    return recs

In [10]:
def make_objective_svd( val_items, all_items,
                       metric_fn=ndcg_at_k, k_eval=10):
    """
    Objective-функция для Optuna: подбор параметров SVD.
    """

    def objective(trial):
        # число компонент
        n_components = trial.suggest_int("n_components", 1, 250, log=True)
        n_iter = trial.suggest_int("n_iter", 1, 250, log=True)
        center = trial.suggest_categorical("center", [None, 'user' , 'item' , 'both'])
        normalize = trial.suggest_categorical("normalize", [None, 'zscore' , 'minmax'])


        v_explicit_train_matrix, _, _ =\
            prepare_ui_matrix(
                          v_train,
                          user_col='user_id',
                          item_col='item_id',
                          rating_col='rating',
                          implicit=False,       # работаем с explicit рейтингами
                          threshold=None,       # не фильтруем по порогу что бы матрица
                                                # была соразмерна implisit матрице
                          center= center,       # подбираем
                          normalize=normalize   # подбираем
                      )

        # создаём модель
        svd = TruncatedSVD(n_components=n_components, random_state=RANDOM_STATE, n_iter=n_iter )
        svd.fit(v_explicit_train_matrix)

        # рекомендации
        rec_items = recommend_with_svd(svd, v_explicit_train_matrix, list(val_items.keys()), topn=k_eval)

        # метрика
        score = metric_fn(rec_items, val_items, k=k_eval)

        # print(f'\ntrial SVD with: n_components={n_components} ... , score={score}\n')
        return score

    return objective

In [11]:
n_trials=50
# замерим время
start = time.time()

sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)

# truncated_SVD
study_svd = optuna.create_study(direction="maximize", sampler=sampler)
study_svd.optimize(
    make_objective_svd( val_items_ex, all_items_ex,
                      metric_fn=ndcg_at_k, k_eval=10),
    n_trials=n_trials
)

end = time.time()
study_svd_time = end - start

print('#'*70)
print("\nЛучшие параметры truncated:", study_svd.best_params)
print("Лучший score:", study_svd.best_value)

print('\n')
print('*'*70)
print(f"Время выполнения: {study_svd_time:.2f} секунд")

[I 2025-11-17 16:37:27,882] A new study created in memory with name: no-name-d13aeb68-a814-45ab-8d84-49f1649b3444
[I 2025-11-17 16:37:32,925] Trial 0 finished with value: 0.3531223286186831 and parameters: {'n_components': 5, 'n_iter': 184, 'center': None, 'normalize': 'zscore'}. Best is trial 0 with value: 0.3531223286186831.
[I 2025-11-17 16:37:33,598] Trial 1 finished with value: 0.2734215840894567 and parameters: {'n_components': 41, 'n_iter': 1, 'center': None, 'normalize': 'minmax'}. Best is trial 0 with value: 0.3531223286186831.
[I 2025-11-17 16:37:34,211] Trial 2 finished with value: 0.3538465704640838 and parameters: {'n_components': 7, 'n_iter': 3, 'center': None, 'normalize': 'zscore'}. Best is trial 2 with value: 0.3538465704640838.
[I 2025-11-17 16:37:34,930] Trial 3 finished with value: 0.3329606270720822 and parameters: {'n_components': 12, 'n_iter': 20, 'center': 'user', 'normalize': 'zscore'}. Best is trial 2 with value: 0.3538465704640838.
[I 2025-11-17 16:37:35,165]

######################################################################

Лучшие параметры truncated: {'n_components': 5, 'n_iter': 42, 'center': None, 'normalize': None}
Лучший score: 0.35495736618448653


**********************************************************************
Время выполнения: 55.05 секунд


In [12]:
# optuna визуализация
print("OPTUNA for item_KNN")
optuna.visualization.plot_optimization_history(study_svd).show()
optuna.visualization.plot_param_importances(study_svd).show()
optuna.visualization.plot_parallel_coordinate(study_svd).show()

OPTUNA for item_KNN


### Выводы:
**Optuna - молодец**
- помогла выбрать параметры для результата гораздо выше среднего  

**Лучшие параметры**  
- n_components = 5
- n_iter ≈ 42
- center = None
- normalize = None
- Score ≈ 0.355 по NDCG@10   

Оптимум достигается при малой размерности латентного пространства и умеренном числе итераций.   

---
**Роль параметров**

**n_components**
- Оптимум в диапазоне 5-7.
- При росте числа компонент качество падает

**n_iter**
- Слишком маленькие значения (1-3) не успевают обучить модель.
- Слишком большие  не дают прироста.
- Оптимум - средний диапазон (30-50).

**center**

- Лучшие результаты при отсутствии центрирования.

**normalize**

- Лучшие результаты при None.
- zscore и minmax чаще ухудшают NDCG.

## 06. Получение метрик на test

In [ ]:
## Получение метрик на test
explicit_train_matrix, _, _ =\
    prepare_ui_matrix(
                  train,
                  user_col='user_id',
                  item_col='item_id',
                  rating_col='rating',
                  implicit=False,       # работаем с explicit рейтингами
                  threshold=None,       # не фильтруем по порогу что бы матрица
                  center= None,
                  normalize=None
              )

In [ ]:
# загрузка
input_dir = PROCESSED/"artifacts"

# Загрузка словарей
with open(input_dir / "user2index.pkl", "rb") as f:
    user2index = pickle.load(f)

with open(input_dir / "item2index.pkl", "rb") as f:
    item2index = pickle.load(f)

with open(input_dir / "index2user.pkl", "rb") as f:
    index2user = pickle.load(f)

with open(input_dir / "index2item.pkl", "rb") as f:
    index2item = pickle.load(f)

In [ ]:
# создаём модель
svd = TruncatedSVD(n_components=5, random_state=RANDOM_STATE, n_iter=42 )
svd.fit(explicit_train_matrix)

TruncatedSVD(n_components=5, n_iter=42, random_state=42)

In [ ]:
# заменяем реальные ID на индексы
test_mapped = test.assign(
    user_id = test["user_id"].map(user2index),
    item_id = test["item_id"].map(item2index)
)
assert test_mapped.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
test_items = test_mapped.groupby('user_id')['item_id'].apply(set).to_dict()

# all_items
all_items = set(train['item_id'].map(item2index).dropna().astype(int).unique())

In [ ]:
# рекомендации
svd_recs = recommend_with_svd(svd, explicit_train_matrix, list(test_items.keys()), topn=TOP_N)
svd_results = model_evaluation(svd_recs,test_items, all_items, 10, 'trancated_svd_n_comp=5_n_iter=42')
svd_results

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
trancated_svd_n_comp=5_n_iter=42,0.851675,0.348206,0.045528,0.365538,0.24731,0.091207


### Сохраняем результаты эксперимента

In [ ]:
results_data, json_file, csv_file = save_experiment_results(
    result=svd_results,
    model_name="trancated_svd_n_comp=5_n_iter=42",
    meta=meta,
    results_dir = RESULTS_DIR
)

Результат добавлен в существующий CSV файл
JSON результат сохранен как: trancated_svd_n_comp=5_n_iter=42_20251116_190936.json
CSV со всеми экспериментами: all_experiments_results.csv
Все результаты в: /content/drive/MyDrive/Colab Notebooks/data/results

СВОДКА ЭКСПЕРИМЕНТА
Модель: trancated_svd_n_comp=5_n_iter=42
Метка времени: 20251116_190936
Дата оценки: 2025-11-16T19:09:36
Размер train: 800,142
Размер test: 94,842
Пользователей в test: 836
Уникальных предметов: 3662
HitRate@10: 85.2%
precision@10: 34.82%
recall@10: 4.55%
ndcg@10: 36.55%
map@10: 24.73%
Coverage@10: 9.12%

Последние эксперименты (4 всего):


,model_name,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10,timestamp,evaluation_date
0,Most_Popular,0.515550,0.132895,0.015591,0.133171,0.068656,0.002731,20251022_194247,2025-10-22T19:42:47.576490
1,itemKNN_tfidf_k=312,0.812201,0.329067,0.040669,0.345263,0.233586,0.074823,20251108_084840,2025-11-08T08:48:40.783074
2,userKNN_bmp25_k=991,0.836124,0.341029,0.043678,0.361278,0.244691,0.063900,20251108_084943,2025-11-08T08:49:43.731391
3,trancated_svd_n_comp=5_n_iter=42,0.851675,0.348206,0.045528,0.365538,0.247310,0.091207,20251116_190936,2025-11-16T19:09:36.863512


# **Итого:**

## **наблюдения:**
**Truncated SVD (n=5)**
- новый чемпион ! показал лучшие результаты по всем ключевым метрикам:
- **HitRate@10**: 85%
- **Precision@10**: 34.8% (чуть выше userKNN).
- **Recall@10:** 4.55% (самый высокий среди всех).
- **NDCG@10** и **MAP@10** тоже максимальные.

- **Coverage@10**: 9.1% - заметно выше.

**KNN-модели** (itemKNN и userKNN) тоже сильны, но уступают SVD на всех метриках.

- **userKNN** чуть лучше itemKNN по precision/recall, но хуже по coverage.

- **itemKNN** даёт более широкое покрытие, но чуть слабее по качеству рекомендаций.

- **Most Popular** ожидаемо слабый baseline
## **Выводы**:
- **Truncated SVD** при небольшом числе компонент (n=5) оказался оптимальным.

- **КNN-подходы** остаются конкурентными, но проигрывают по coverage и чуть по метрикам качества.

- baseline **"Most Popular"** сильно уступает, что подтверждает ценность факторизационных и соседских методов.

Интересно, что SVD выигрывает именно на маленьком числе компонент - это может означать, что данные хорошо описываются несколькими глобальными латентными факторами, а увеличение размерности приводит к переобучению или шуму.

# '######################################################'